In [ ]:
!pip install imutils

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import cv2
import pydicom
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
image = pydicom.dcmread('/kaggle/input/siim-covid19-detection/train/08bf9775ebfd/618c811827ef/f43ef9430a14.dcm')
#print(image)
plt.figure(figsize=(10,10))
plt.imshow(image.pixel_array, cmap=plt.cm.bone)
plt.show()

In [ ]:
image.pixel_array.shape

In [ ]:
image = image.pixel_array

In [ ]:
thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(thresh, cmap=plt.cm.bone)
plt.show()

In [ ]:
class SimplePreprocessor:
    def __init__(self, width, height, inter=cv2.INTER_AREA):
        self.width = width
        self.height = height
        self.inter = inter
    
    def proprocess(self, image):
        return cv2.resize(image, (self.width, self.height), interpolation=self.inter)
        
    def imagethresh(self, image):
        thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        return thresh

In [ ]:
class SimpleDatasetLoader:
    def __init__(self, preprocessors=None):
        self.preprocessors = preprocessors
        
        if self.preprocessors  is None:
            self.preprocessors = []
            
    def load(self, imagePaths, verbose=-1):
        data =[]
        labels = []
        
        for (i, imagePath) in enumerate(imagePaths):
            if imagePath.split(".")[-1] == 'dcm':
                image = pydicom.dcmread(imagePath)
                image = image.pixel_array            
            
            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.proprocess(image)
                    image = p.imagethresh(image)
                    
            data.append(image)
            
            if i >15:
                break
            
            if verbose > 0 and  i > 0 and (i+1)% verbose ==0:
                print("[INFO] processed {} /{}".format(i+1, len(imagePaths)))
            
        return (np.array(data))
            

In [ ]:
print("[INFO] loading image ...")
imagePaths = list(paths.list_files('/kaggle/input/siim-covid19-detection/train/'))  

sp = SimplePreprocessor(64, 64)
sdl = SimpleDatasetLoader(preprocessors=[sp])
data = sdl.load(imagePaths, verbose=100)
data = data.reshape((data.shape[0], 4096))